<a href="https://colab.research.google.com/github/hemasri9391/Adobe_Round-1b/blob/main/Round_1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF
!pip install nltk
import nltk
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 39.8 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import fitz
import requests
from nltk.tokenize import sent_tokenize
from datetime import datetime
import json

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.4 MB/s eta 0:00:00


In [ ]:
pdf_urls = [
    "https://arxiv.org/pdf/2103.09430.pdf",
    "https://arxiv.org/pdf/2204.08706.pdf",
    "https://arxiv.org/pdf/2006.01460.pdf"
]

pdf_docs = []
for idx, url in enumerate(pdf_urls):
    response = requests.get(url)
    filename = f"paper_{idx+1}.pdf"
    with open(filename, "wb") as f:
        f.write(response.content)
    pdf_docs.append(filename)
print("All PDFs downloaded successfully.")

All PDFs downloaded successfully.


In [ ]:
from IPython.display import display, Markdown
import requests
from PyPDF2 import PdfReader
from io import BytesIO
def extract_title_from_url(url):
    response = requests.get(url)
    reader = PdfReader(BytesIO(response.content))
    first_page = reader.pages[0]
    text = first_page.extract_text()
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    title_candidates = [line for line in lines if 20 < len(line) < 1000]
    title = title_candidates[0] if title_candidates else "Title not found"
    return title

display(Markdown("**Extracted Research Paper Titles**"))

for idx, url in enumerate(pdf_urls):
    title = extract_title_from_url(url)
    display(Markdown(f"Paper {idx + 1}:\n **{title}**"))


**Extracted Research Paper Titles**

Paper 1:
 **OGB-LSC: A Large-Scale Challenge for**

Paper 2:
 **Recycling End–of–life**

Paper 3:
 **Situated and Interactive Multimodal Conversations**

In [ ]:
def extract_sections_from_pdf(file_path):
    doc = fitz.open(file_path)
    extracted = []

    for page_num, page in enumerate(doc, start=1):
        text = page.get_text()
        if any(keyword in text.lower() for keyword in ["method", "dataset", "benchmark"]):
            extracted.append({
                "document": file_path,
                "page_number": page_num,
                "section_title": f"Possible Relevant Section Page {page_num}",
                "importance_rank": 1 if "method" in text.lower() else 2
            })
    return extracted

all_extracted_sections = []
for pdf in pdf_docs:
    extracted = extract_sections_from_pdf(pdf)
    all_extracted_sections.extend(extracted)
print(f"Extracted {len(all_extracted_sections)} relevant sections.")

Extracted 40 relevant sections.


In [ ]:
import nltk
nltk.download('punkt_tab')
def get_refined_text(file_path, page_num):
    doc = fitz.open(file_path)
    page_text = doc[page_num - 1].get_text()
    sentences = sent_tokenize(page_text)
    refined = " ".join(sentences[:3])  # taking first 3 sentences for demo
    return refined

subsection_analysis = []
for section in all_extracted_sections:
    refined_text = get_refined_text(section["document"], section["page_number"])
    subsection_analysis.append({
        "document": section["document"],
        "page_number": section["page_number"],
        "refined_text": refined_text
    })

print(f"Sub-section analysis for {len(subsection_analysis)} sections done.")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Sub-section analysis for 40 sections done.


In [ ]:
output = {
    "metadata": {
        "input_documents": pdf_docs,
        "persona": "PhD Researcher in Computational Biology",
        "job_to_be_done": "Prepare a comprehensive literature review focusing on methodologies, datasets, and performance benchmarks",
        "processing_timestamp": datetime.now().isoformat()
    },
    "extracted_sections": all_extracted_sections,
    "subsection_analysis": subsection_analysis
}
print(json.dumps(output, indent=2))

{
  "metadata": {
    "input_documents": [
      "paper_1.pdf",
      "paper_2.pdf",
      "paper_3.pdf"
    ],
    "persona": "PhD Researcher in Computational Biology",
    "job_to_be_done": "Prepare a comprehensive literature review focusing on methodologies, datasets, and performance benchmarks",
    "processing_timestamp": "2025-07-22T14:22:10.808977"
  },
  "extracted_sections": [
    {
      "document": "paper_1.pdf",
      "page_number": 1,
      "section_title": "Possible Relevant Section Page 1",
      "importance_rank": 2
    },
    {
      "document": "paper_1.pdf",
      "page_number": 2,
      "section_title": "Possible Relevant Section Page 2",
      "importance_rank": 1
    },
    {
      "document": "paper_1.pdf",
      "page_number": 3,
      "section_title": "Possible Relevant Section Page 3",
      "importance_rank": 2
    },
    {
      "document": "paper_1.pdf",
      "page_number": 4,
      "section_title": "Possible Relevant Section Page 4",
      "importance_ran